In [1]:
import datasets
import polars as pl
import pandas as pd
import numpy as np

from tqdm import tqdm
from sentence_transformers import SentenceTransformer

tqdm.pandas()

SEED = 69

np.random.seed(SEED)

/home/v_larkin/.pyenv/versions/3.12.2/envs/vvv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
vacancies = pl.read_parquet("hh_recsys_vacancies.pq").to_pandas()

In [3]:
vacancies

,vacancy_id,name,company.id,description,keySkills.keySkill,compensation.from,compensation.to,compensation.currencyCode,area.id,area.regionId,employment,workSchedule,workExperience
0,v_862116,Смотритель музейный,c_162972,<strong>Обязанности:</strong> <ul> <li>Осущест...,"[Пользователь ПК, Работа в команде, Умение пла...",16500.0,NaN,RUR,a_4761,ar_33,full,fullDay,noExperience
1,v_288642,Ведущий менеджер по работе с физическими лицами,c_208672,"<p><strong>Возможно, наша вакансия именно та, ...","[Активные продажи, Холодные продажи, Кредитные...",50000.0,NaN,RUR,a_744,ar_2,full,fullDay,noExperience
2,v_1840054,Бухгалтер (по расчету зарплаты),c_198109,<strong>Обязанности:</strong> <ul> <li>Расчет ...,None,50000.0,65000.0,RUR,a_6223,ar_78,full,fullDay,between3And6
3,v_2346232,"Пекарь (Токсово, Привокзальная, 16)",c_6137,"<p><strong>Для каждого, кто хочет работать и з...",None,38500.0,42000.0,RUR,a_4795,ar_51,full,fullDay,noExperience
4,v_312507,Торговый представитель (г. Абакан),c_206699,<p>Компания ТД &quot;Оливьера&quot;- лучший ди...,"[Продуктивность, Клиентоориентированность, Упр...",60000.0,NaN,RUR,a_6837,ar_4,full,fullDay,between1And3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2734124,v_2484959,Сборщик-упаковщик,c_203256,<p><strong>Обязанности:</strong></p> <ul> <li>...,"[Пользователь ПК, Умение работать в команде, У...",40000.0,90000.0,RUR,a_5387,ar_71,full,flexible,noExperience
2734125,v_205163,Сварщик на полуавтомат,c_158695,<p>ОКЛАД 80 000 РУБЛЕЙ. В связи с увеличением ...,"[Желание работать и зарабатывать, Высокая энер...",80000.0,130000.0,RUR,a_5527,ar_69,full,fullDay,between1And3
2734126,v_639897,Главный инженер / Технический директор,c_209365,<p>Компания<strong> МОБИЛЬНЫЙ ДОМ - ЛИДЕР В СФ...,"[Контроль исправности оборудования, Инженерные...",200000.0,NaN,RUR,a_1756,ar_41,full,fullDay,between1And3
2734127,v_1636531,"Провизор/Фармацевт (г.Адыгейск, 20 км от Красн...",c_246244,<strong>Требования:</strong> <ul> <li> <p>Зако...,"[Предпечатная подготовка, Статистический анализ]",25000.0,NaN,RUR,a_3403,ar_60,full,fullDay,noExperience


In [4]:
vacancies["compensation.currencyCode"].unique()

array(['RUR', None, 'USD', 'EUR', 'KZT', 'BYR', 'UZS', 'KGS', 'UAH',
       'GEL', 'AZN'], dtype=object)

In [5]:
vacancies["workSchedule"].unique()

array(['fullDay', 'flyInFlyOut', 'flexible', 'shift', 'remote'],
      dtype=object)

In [6]:
vacancies["workExperience"].unique()

array(['noExperience', 'between3And6', 'between1And3', 'moreThan6'],
      dtype=object)

In [7]:
text = vacancies["description"][4]
text

'<p>Компания ТД &quot;Оливьера&quot;- лучший дистрибьютор диетического питания на территории Красноярского края! Наше призвание- создание условий для более качественной и <em>здоров</em>ой жизни людей! Мы раскрываем ценность нашей продукции максимально большему количеству покупателей! Стремимся создавать комфортные условия работы, в которых каждый будет чувствовать себя нужным и важным!</p> <p><strong>Что нужно будет делать:</strong></p> <ul> <li>Привлекать клиентов, заключать договора, расширять клиентскую базу (опт, розница);</li> <li>Расширять ассортимент в торговых точках, увеличивать объём продаж;</li> <li>Работать с существующими клиентами, с акциями, документами;</li> <li>Вести дебиторскую задолженность клиентов, сводя её к нулю.</li> </ul> <p><strong>Мы будем рады рассмотреть твою кандидатуру, если у тебя есть:</strong></p> <ul> <li>Приветствуется <em>опыт</em> работы торговым представителем (продукты питания, бакалея);</li> <li><em>Опыт</em> успешных продаж (увеличил(а) клиент

In [8]:
import re

html_pattern = re.compile(r"(?:\<\/?.+?\>)")  # регулярка на HTML-теги
re.findall(html_pattern, text)

['<p>',
 '<em>',
 '</em>',
 '</p>',
 '<p>',
 '<strong>',
 '</strong>',
 '</p>',
 '<ul>',
 '<li>',
 '</li>',
 '<li>',
 '</li>',
 '<li>',
 '</li>',
 '<li>',
 '</li>',
 '</ul>',
 '<p>',
 '<strong>',
 '</strong>',
 '</p>',
 '<ul>',
 '<li>',
 '<em>',
 '</em>',
 '</li>',
 '<li>',
 '<em>',
 '</em>',
 '</li>',
 '<li>',
 '<em>',
 '</em>',
 '</li>',
 '<li>',
 '<em>',
 '</em>',
 '</li>',
 '<li>',
 '</li>',
 '</ul>',
 '<p>',
 '<strong>',
 '</strong>',
 '</p>',
 '<ul>',
 '<li>',
 '</li>',
 '<li>',
 '<em>',
 '</em>',
 '</li>',
 '<li>',
 '</li>',
 '<li>',
 '</li>',
 '<li>',
 '</li>',
 '<li>',
 '<em>',
 '</em>',
 '</li>',
 '<li>',
 '</li>',
 '<li>',
 '</li>',
 '</ul>']

In [9]:
re.sub(html_pattern, "", text)  # удаляем HTML-теги

'Компания ТД &quot;Оливьера&quot;- лучший дистрибьютор диетического питания на территории Красноярского края! Наше призвание- создание условий для более качественной и здоровой жизни людей! Мы раскрываем ценность нашей продукции максимально большему количеству покупателей! Стремимся создавать комфортные условия работы, в которых каждый будет чувствовать себя нужным и важным! Что нужно будет делать:  Привлекать клиентов, заключать договора, расширять клиентскую базу (опт, розница); Расширять ассортимент в торговых точках, увеличивать объём продаж; Работать с существующими клиентами, с акциями, документами; Вести дебиторскую задолженность клиентов, сводя её к нулю.  Мы будем рады рассмотреть твою кандидатуру, если у тебя есть:  Приветствуется опыт работы торговым представителем (продукты питания, бакалея); Опыт успешных продаж (увеличил(а) клиентскую базу, товарооборот); Выстроенные долгосрочные отношения с лицами, принимающими решения в торговых точках; Опыт работы с дебиторской задолже

In [10]:
# удалим теги и лишние пробелы

vacancies["description"] = vacancies["description"].progress_apply(lambda t: t.replace("&quot;", '"').replace("</p>", "\n").replace("<ul>", "\n").replace("</li>", "\n").replace("<li>", "- "))
vacancies["description"] = vacancies["description"].progress_apply(lambda t: re.sub(html_pattern, "", t))
vacancies["description"] = vacancies["description"].progress_apply(lambda t: re.sub(r"\s{3,}", " ", t))
vacancies["description"] = vacancies["description"].progress_apply(lambda t: re.sub(r"(?:\&.+?\;)", "", t).strip())

vacancies.head()

100%|██████████| 2734129/2734129 [00:03<00:00, 686847.43it/s]


,vacancy_id,name,company.id,description,keySkills.keySkill,compensation.from,compensation.to,compensation.currencyCode,area.id,area.regionId,employment,workSchedule,workExperience
0,v_862116,Смотритель музейный,c_162972,Обязанности: - Осуществлять контроль за соблюд...,"[Пользователь ПК, Работа в команде, Умение пла...",16500.0,NaN,RUR,a_4761,ar_33,full,fullDay,noExperience
1,v_288642,Ведущий менеджер по работе с физическими лицами,c_208672,"Возможно, наша вакансия именно та, которую ты ...","[Активные продажи, Холодные продажи, Кредитные...",50000.0,NaN,RUR,a_744,ar_2,full,fullDay,noExperience
2,v_1840054,Бухгалтер (по расчету зарплаты),c_198109,Обязанности: - Расчет заработной платы (в том ...,None,50000.0,65000.0,RUR,a_6223,ar_78,full,fullDay,between3And6
3,v_2346232,"Пекарь (Токсово, Привокзальная, 16)",c_6137,"Для каждого, кто хочет работать и зарабатывать...",None,38500.0,42000.0,RUR,a_4795,ar_51,full,fullDay,noExperience
4,v_312507,Торговый представитель (г. Абакан),c_206699,"Компания ТД ""Оливьера""- лучший дистрибьютор ди...","[Продуктивность, Клиентоориентированность, Упр...",60000.0,NaN,RUR,a_6837,ar_4,full,fullDay,between1And3


In [11]:
# склеим все текстовые поля в одно общее описание

full_text = []
for _, row in tqdm(vacancies.iterrows(), total=vacancies.shape[0]):
    text = f"Вакансия: {row["name"].strip()}"
    if row["keySkills.keySkill"] is not None:
        text += f"\nКлючевые навыки: {", ".join(row['keySkills.keySkill'])}"

    text += f"\n\n{row["description"]}"
    full_text.append(text)

vacancies["full_text"] = full_text
vacancies.head()

100%|██████████| 2734129/2734129 [00:46<00:00, 59134.26it/s]


,vacancy_id,name,company.id,description,keySkills.keySkill,compensation.from,compensation.to,compensation.currencyCode,area.id,area.regionId,employment,workSchedule,workExperience,full_text
0,v_862116,Смотритель музейный,c_162972,Обязанности: - Осуществлять контроль за соблюд...,"[Пользователь ПК, Работа в команде, Умение пла...",16500.0,NaN,RUR,a_4761,ar_33,full,fullDay,noExperience,Вакансия: Смотритель музейный\nКлючевые навыки...
1,v_288642,Ведущий менеджер по работе с физическими лицами,c_208672,"Возможно, наша вакансия именно та, которую ты ...","[Активные продажи, Холодные продажи, Кредитные...",50000.0,NaN,RUR,a_744,ar_2,full,fullDay,noExperience,Вакансия: Ведущий менеджер по работе с физичес...
2,v_1840054,Бухгалтер (по расчету зарплаты),c_198109,Обязанности: - Расчет заработной платы (в том ...,None,50000.0,65000.0,RUR,a_6223,ar_78,full,fullDay,between3And6,Вакансия: Бухгалтер (по расчету зарплаты)\n\nО...
3,v_2346232,"Пекарь (Токсово, Привокзальная, 16)",c_6137,"Для каждого, кто хочет работать и зарабатывать...",None,38500.0,42000.0,RUR,a_4795,ar_51,full,fullDay,noExperience,"Вакансия: Пекарь (Токсово, Привокзальная, 16)\..."
4,v_312507,Торговый представитель (г. Абакан),c_206699,"Компания ТД ""Оливьера""- лучший дистрибьютор ди...","[Продуктивность, Клиентоориентированность, Упр...",60000.0,NaN,RUR,a_6837,ar_4,full,fullDay,between1And3,Вакансия: Торговый представитель (г. Абакан)\n...


In [12]:
print(vacancies["full_text"][4])

Вакансия: Торговый представитель (г. Абакан)
Ключевые навыки: Продуктивность, Клиентоориентированность, Управление отношениями с клиентами, Деловая коммуникация, Поиск и привлечение клиентов, Активные продажи, Развитие продаж

Компания ТД "Оливьера"- лучший дистрибьютор диетического питания на территории Красноярского края! Наше призвание- создание условий для более качественной и здоровой жизни людей! Мы раскрываем ценность нашей продукции максимально большему количеству покупателей! Стремимся создавать комфортные условия работы, в которых каждый будет чувствовать себя нужным и важным!
 Что нужно будет делать: - Привлекать клиентов, заключать договора, расширять клиентскую базу (опт, розница);
 - Расширять ассортимент в торговых точках, увеличивать объём продаж;
 - Работать с существующими клиентами, с акциями, документами;
 - Вести дебиторскую задолженность клиентов, сводя её к нулю. Мы будем рады рассмотреть твою кандидатуру, если у тебя есть: - Приветствуется опыт работы торговым п

In [13]:
# model = SentenceTransformer("efederici/sentence-bert-base").cuda()
# embeddings = model.encode(vacancies["full_text"][:5], show_progress_bar=True)
# embeddings

In [14]:
# добавим эмбеддинги sentence-bert для полного описания вакансий

# embeddings = model.encode(vacancies["full_text"], show_progress_bar=True, batch_size=400)

# np.save("embeddings", embeddings, allow_pickle=False)

In [15]:
embeddings = np.load("embeddings.npy")

In [16]:
vacancies["embedding"] = [e for e in embeddings]
vacancies.head()

,vacancy_id,name,company.id,description,keySkills.keySkill,compensation.from,compensation.to,compensation.currencyCode,area.id,area.regionId,employment,workSchedule,workExperience,full_text,embedding
0,v_862116,Смотритель музейный,c_162972,Обязанности: - Осуществлять контроль за соблюд...,"[Пользователь ПК, Работа в команде, Умение пла...",16500.0,NaN,RUR,a_4761,ar_33,full,fullDay,noExperience,Вакансия: Смотритель музейный\nКлючевые навыки...,"[-1.179423, 0.18333192, -0.33118993, 0.4968867..."
1,v_288642,Ведущий менеджер по работе с физическими лицами,c_208672,"Возможно, наша вакансия именно та, которую ты ...","[Активные продажи, Холодные продажи, Кредитные...",50000.0,NaN,RUR,a_744,ar_2,full,fullDay,noExperience,Вакансия: Ведущий менеджер по работе с физичес...,"[-1.0414252, -0.04030051, -0.20509179, 0.67496..."
2,v_1840054,Бухгалтер (по расчету зарплаты),c_198109,Обязанности: - Расчет заработной платы (в том ...,None,50000.0,65000.0,RUR,a_6223,ar_78,full,fullDay,between3And6,Вакансия: Бухгалтер (по расчету зарплаты)\n\nО...,"[-1.467348, 0.46711677, -0.2660433, 0.4765512,..."
3,v_2346232,"Пекарь (Токсово, Привокзальная, 16)",c_6137,"Для каждого, кто хочет работать и зарабатывать...",None,38500.0,42000.0,RUR,a_4795,ar_51,full,fullDay,noExperience,"Вакансия: Пекарь (Токсово, Привокзальная, 16)\...","[-1.5293988, 0.3218064, -0.13066131, 0.5563953..."
4,v_312507,Торговый представитель (г. Абакан),c_206699,"Компания ТД ""Оливьера""- лучший дистрибьютор ди...","[Продуктивность, Клиентоориентированность, Упр...",60000.0,NaN,RUR,a_6837,ar_4,full,fullDay,between1And3,Вакансия: Торговый представитель (г. Абакан)\n...,"[-1.478767, 0.32001862, -0.12730715, 0.6519245..."


In [17]:
emb_mean = np.mean(embeddings)
emb_std = np.std(embeddings)

print(float(emb_mean))
print(float(emb_std))

-0.017310157418251038
0.5394255518913269


In [18]:
# нормализуем эмбеддинги, чтобы было mean=0, std=1

mean = -0.017310157418251038
std = 0.5394255518913269

embeddings_normilized = (embeddings - mean) / std

embeddings_normilized.mean(), embeddings_normilized.std()

(-7.171174e-07, 0.9999959)

In [19]:
vacancies["embedding"] = [e for e in embeddings_normilized]
vacancies.head()

,vacancy_id,name,company.id,description,keySkills.keySkill,compensation.from,compensation.to,compensation.currencyCode,area.id,area.regionId,employment,workSchedule,workExperience,full_text,embedding
0,v_862116,Смотритель музейный,c_162972,Обязанности: - Осуществлять контроль за соблюд...,"[Пользователь ПК, Работа в команде, Умение пла...",16500.0,NaN,RUR,a_4761,ar_33,full,fullDay,noExperience,Вакансия: Смотритель музейный\nКлючевые навыки...,"[-2.1543527, 0.3719551, -0.5818779, 0.95323056..."
1,v_288642,Ведущий менеджер по работе с физическими лицами,c_208672,"Возможно, наша вакансия именно та, которую ты ...","[Активные продажи, Холодные продажи, Кредитные...",50000.0,NaN,RUR,a_744,ar_2,full,fullDay,noExperience,Вакансия: Ведущий менеджер по работе с физичес...,"[-1.898529, -0.042620067, -0.34811407, 1.28335..."
2,v_1840054,Бухгалтер (по расчету зарплаты),c_198109,Обязанности: - Расчет заработной платы (в том ...,None,50000.0,65000.0,RUR,a_6223,ar_78,full,fullDay,between3And6,Вакансия: Бухгалтер (по расчету зарплаты)\n\nО...,"[-2.688115, 0.8980422, -0.46110746, 0.91553205..."
3,v_2346232,"Пекарь (Токсово, Привокзальная, 16)",c_6137,"Для каждого, кто хочет работать и зарабатывать...",None,38500.0,42000.0,RUR,a_4795,ar_51,full,fullDay,noExperience,"Вакансия: Пекарь (Токсово, Привокзальная, 16)\...","[-2.8031461, 0.6286624, -0.21013308, 1.0635489..."
4,v_312507,Торговый представитель (г. Абакан),c_206699,"Компания ТД ""Оливьера""- лучший дистрибьютор ди...","[Продуктивность, Клиентоориентированность, Упр...",60000.0,NaN,RUR,a_6837,ar_4,full,fullDay,between1And3,Вакансия: Торговый представитель (г. Абакан)\n...,"[-2.7092838, 0.6253482, -0.20391504, 1.2406433..."


In [20]:
# посмотрим что осталось обработать

vacancies = vacancies.drop(columns=["name", "description", "keySkills.keySkill", "full_text"])
vacancies.head()

,vacancy_id,company.id,compensation.from,compensation.to,compensation.currencyCode,area.id,area.regionId,employment,workSchedule,workExperience,embedding
0,v_862116,c_162972,16500.0,NaN,RUR,a_4761,ar_33,full,fullDay,noExperience,"[-2.1543527, 0.3719551, -0.5818779, 0.95323056..."
1,v_288642,c_208672,50000.0,NaN,RUR,a_744,ar_2,full,fullDay,noExperience,"[-1.898529, -0.042620067, -0.34811407, 1.28335..."
2,v_1840054,c_198109,50000.0,65000.0,RUR,a_6223,ar_78,full,fullDay,between3And6,"[-2.688115, 0.8980422, -0.46110746, 0.91553205..."
3,v_2346232,c_6137,38500.0,42000.0,RUR,a_4795,ar_51,full,fullDay,noExperience,"[-2.8031461, 0.6286624, -0.21013308, 1.0635489..."
4,v_312507,c_206699,60000.0,NaN,RUR,a_6837,ar_4,full,fullDay,between1And3,"[-2.7092838, 0.6253482, -0.20391504, 1.2406433..."


In [21]:
vacancies["company.id"] = vacancies["company.id"].progress_apply(lambda t: int(t.removeprefix("c_")))
vacancies["area.id"] = vacancies["area.id"].progress_apply(lambda t: int(t.removeprefix("a_")))
vacancies["area.regionId"] = vacancies["area.regionId"].progress_apply(lambda t: int(t.removeprefix("ar_")) if t is not None else None)

vacancies.head()

100%|██████████| 2734129/2734129 [00:01<00:00, 2457029.65it/s]


,vacancy_id,company.id,compensation.from,compensation.to,compensation.currencyCode,area.id,area.regionId,employment,workSchedule,workExperience,embedding
0,v_862116,162972,16500.0,NaN,RUR,4761,33.0,full,fullDay,noExperience,"[-2.1543527, 0.3719551, -0.5818779, 0.95323056..."
1,v_288642,208672,50000.0,NaN,RUR,744,2.0,full,fullDay,noExperience,"[-1.898529, -0.042620067, -0.34811407, 1.28335..."
2,v_1840054,198109,50000.0,65000.0,RUR,6223,78.0,full,fullDay,between3And6,"[-2.688115, 0.8980422, -0.46110746, 0.91553205..."
3,v_2346232,6137,38500.0,42000.0,RUR,4795,51.0,full,fullDay,noExperience,"[-2.8031461, 0.6286624, -0.21013308, 1.0635489..."
4,v_312507,206699,60000.0,NaN,RUR,6837,4.0,full,fullDay,between1And3,"[-2.7092838, 0.6253482, -0.20391504, 1.2406433..."


In [22]:
region_id = vacancies["area.regionId"]
region_id.min(), region_id.max()

(0.0, 104.0)

In [23]:
reg_min = 0
reg_max = 104

vacancies["area.regionId"] = (vacancies["area.regionId"] - reg_min) * 2 / reg_max - 1  # min-max scaling в диапазон [-1, 1]
vacancies["area.regionId"].describe()

count    2.712786e+06
mean     1.776757e-03
std      5.131591e-01
min     -1.000000e+00
25%     -2.115385e-01
50%     -1.538462e-01
75%      4.230769e-01
max      1.000000e+00
Name: area.regionId, dtype: float64

In [24]:
nan_reg_ids = np.random.uniform(-1, 1, size=vacancies[vacancies["area.regionId"].isna()].shape[0])
nan_reg_ids, nan_reg_ids.shape

(array([-0.40750168,  0.61813543, -0.29949495, ..., -0.0921034 ,
        -0.89432375, -0.35829205]),
 (21343,))

In [25]:
# заменим nan в regionID на рандомные (их вроде не так много и шум не должен стать слишком сильным)

vacancies.loc[vacancies["area.regionId"].isna(), "area.regionId"] = nan_reg_ids.tolist()
vacancies[vacancies["area.regionId"].isna()]

,vacancy_id,company.id,compensation.from,compensation.to,compensation.currencyCode,area.id,area.regionId,employment,workSchedule,workExperience,embedding


In [26]:
vacancies.head()

,vacancy_id,company.id,compensation.from,compensation.to,compensation.currencyCode,area.id,area.regionId,employment,workSchedule,workExperience,embedding
0,v_862116,162972,16500.0,NaN,RUR,4761,-0.365385,full,fullDay,noExperience,"[-2.1543527, 0.3719551, -0.5818779, 0.95323056..."
1,v_288642,208672,50000.0,NaN,RUR,744,-0.961538,full,fullDay,noExperience,"[-1.898529, -0.042620067, -0.34811407, 1.28335..."
2,v_1840054,198109,50000.0,65000.0,RUR,6223,0.500000,full,fullDay,between3And6,"[-2.688115, 0.8980422, -0.46110746, 0.91553205..."
3,v_2346232,6137,38500.0,42000.0,RUR,4795,-0.019231,full,fullDay,noExperience,"[-2.8031461, 0.6286624, -0.21013308, 1.0635489..."
4,v_312507,206699,60000.0,NaN,RUR,6837,-0.923077,full,fullDay,between1And3,"[-2.7092838, 0.6253482, -0.20391504, 1.2406433..."


In [27]:
company_id = vacancies["company.id"]
company_id.min(), company_id.max()

(0, 278911)

In [28]:
area_id = vacancies["area.id"]
area_id.min(), area_id.max()

(0, 7013)

In [29]:
company_min = 0
company_max = 278911

area_min = 0
area_max = 7013

vacancies["company.id"] = (vacancies["company.id"] - company_min) * 2 / company_max - 1  # min-max scaling в диапазон [-1, 1]
vacancies["area.id"] = (vacancies["area.id"] - area_min) * 2 / area_max - 1  # min-max scaling в диапазон [-1, 1]

In [30]:
vacancies["company.id"].describe()

count    2.734129e+06
mean    -2.022599e-02
std      5.872816e-01
min     -1.000000e+00
25%     -5.376159e-01
50%     -2.124334e-02
75%      4.832975e-01
max      1.000000e+00
Name: company.id, dtype: float64

In [31]:
vacancies["area.id"].describe()

count    2.734129e+06
mean    -4.582055e-02
std      5.696137e-01
min     -1.000000e+00
25%     -4.992157e-01
50%     -1.772423e-01
75%      5.362897e-01
max      1.000000e+00
Name: area.id, dtype: float64

In [32]:
vacancies.head()

,vacancy_id,company.id,compensation.from,compensation.to,compensation.currencyCode,area.id,area.regionId,employment,workSchedule,workExperience,embedding
0,v_862116,0.168631,16500.0,NaN,RUR,0.357764,-0.365385,full,fullDay,noExperience,"[-2.1543527, 0.3719551, -0.5818779, 0.95323056..."
1,v_288642,0.496334,50000.0,NaN,RUR,-0.787823,-0.961538,full,fullDay,noExperience,"[-1.898529, -0.042620067, -0.34811407, 1.28335..."
2,v_1840054,0.420589,50000.0,65000.0,RUR,0.774704,0.500000,full,fullDay,between3And6,"[-2.688115, 0.8980422, -0.46110746, 0.91553205..."
3,v_2346232,-0.955993,38500.0,42000.0,RUR,0.367460,-0.019231,full,fullDay,noExperience,"[-2.8031461, 0.6286624, -0.21013308, 1.0635489..."
4,v_312507,0.482186,60000.0,NaN,RUR,0.949808,-0.923077,full,fullDay,between1And3,"[-2.7092838, 0.6253482, -0.20391504, 1.2406433..."


In [33]:
vacancies[vacancies["compensation.to"].isna() & vacancies["compensation.from"].isna()]

,vacancy_id,company.id,compensation.from,compensation.to,compensation.currencyCode,area.id,area.regionId,employment,workSchedule,workExperience,embedding
7,v_525733,-0.963422,NaN,NaN,None,0.438186,0.846154,full,fullDay,between3And6,"[-3.0204391, 0.6937193, -0.11872936, 1.2024757..."
17,v_1533950,-0.757643,NaN,NaN,None,-0.787823,-0.961538,full,fullDay,between3And6,"[-2.8537843, 0.7133591, -0.7181119, 1.081198, ..."
27,v_184203,0.506681,NaN,NaN,None,-0.499216,-0.211538,full,flyInFlyOut,between1And3,"[-2.8523123, 0.65945846, -0.4704619, 0.9877909..."
33,v_813145,-0.915651,NaN,NaN,None,-0.499216,-0.211538,full,remote,between1And3,"[-1.388983, 0.06956234, -0.2567434, 0.8853915,..."
38,v_1148119,-0.844990,NaN,NaN,None,-0.499216,-0.211538,full,fullDay,between1And3,"[-1.9793319, 0.11310854, -0.14944212, 1.019585..."
...,...,...,...,...,...,...,...,...,...,...,...
2734093,v_2163321,-0.491960,NaN,NaN,None,-0.499216,-0.211538,full,fullDay,between3And6,"[-2.2244642, 0.5599309, -0.14901, 1.2690829, -..."
2734097,v_1065282,-0.049288,NaN,NaN,None,0.031228,0.903846,full,fullDay,between1And3,"[-2.881256, 0.30403313, -0.510936, 1.2011421, ..."
2734105,v_1527967,0.038668,NaN,NaN,None,-0.499216,-0.211538,full,fullDay,between3And6,"[-1.4246262, 0.033358186, -0.21579592, 0.88232..."
2734107,v_1996112,-0.514605,NaN,NaN,None,-0.499216,-0.211538,full,fullDay,between3And6,"[-1.2897918, 0.16604395, -0.46315512, 0.765063..."


In [34]:
vacancies[vacancies["compensation.to"].isna() & vacancies["compensation.from"].isna() & ~vacancies["compensation.currencyCode"].isna()]

,vacancy_id,company.id,compensation.from,compensation.to,compensation.currencyCode,area.id,area.regionId,employment,workSchedule,workExperience,embedding


In [35]:
compensations = []

# если обе границы по зарплате есть, берём среднее, если только одна, то её, иначе None

for _, row in tqdm(vacancies.iterrows(), total=vacancies.shape[0]):
    comp_from = row["compensation.from"]
    comp_to = row["compensation.to"]
    if comp_from is not None and comp_to is not None:
        compensation = (comp_from + comp_to) / 2
    elif comp_from is not None:
        compensation = comp_from
    elif comp_to is not None:
        compensation = comp_to
    else:
        compensation = None
    compensations.append(compensation)

vacancies["compensation"] = compensations
vacancies.head()

100%|██████████| 2734129/2734129 [00:41<00:00, 65890.75it/s]


,vacancy_id,company.id,compensation.from,compensation.to,compensation.currencyCode,area.id,area.regionId,employment,workSchedule,workExperience,embedding,compensation
0,v_862116,0.168631,16500.0,NaN,RUR,0.357764,-0.365385,full,fullDay,noExperience,"[-2.1543527, 0.3719551, -0.5818779, 0.95323056...",NaN
1,v_288642,0.496334,50000.0,NaN,RUR,-0.787823,-0.961538,full,fullDay,noExperience,"[-1.898529, -0.042620067, -0.34811407, 1.28335...",NaN
2,v_1840054,0.420589,50000.0,65000.0,RUR,0.774704,0.500000,full,fullDay,between3And6,"[-2.688115, 0.8980422, -0.46110746, 0.91553205...",57500.0
3,v_2346232,-0.955993,38500.0,42000.0,RUR,0.367460,-0.019231,full,fullDay,noExperience,"[-2.8031461, 0.6286624, -0.21013308, 1.0635489...",40250.0
4,v_312507,0.482186,60000.0,NaN,RUR,0.949808,-0.923077,full,fullDay,between1And3,"[-2.7092838, 0.6253482, -0.20391504, 1.2406433...",NaN


In [36]:
compensation = vacancies["compensation"]
compensation.min(), compensation.max()

(1.0, 991788366.0)

In [37]:
compensation_min = 1
compensation_max = 991788366

vacancies["compensation"] = (vacancies["compensation"] - compensation_min) * 2 / compensation_max - 1  # min-max scaling в диапазон [-1, 1]
vacancies["compensation"].describe()

count    1.084153e+06
mean    -9.998385e-01
std      1.937615e-03
min     -1.000000e+00
25%     -9.999139e-01
50%     -9.998740e-01
75%     -9.998044e-01
max      1.000000e+00
Name: compensation, dtype: float64

In [38]:
nan_compensation = np.random.uniform(-1, 1, size=vacancies[vacancies["compensation"].isna()].shape[0])
nan_compensation, nan_compensation.shape

(array([-0.49762735,  0.28866506, -0.56903122, ...,  0.52078189,
        -0.39006861,  0.74151366]),
 (1649976,))

In [39]:
vacancies.loc[vacancies["compensation"].isna(), "compensation"] = nan_compensation.tolist()
vacancies[vacancies["compensation"].isna()]

,vacancy_id,company.id,compensation.from,compensation.to,compensation.currencyCode,area.id,area.regionId,employment,workSchedule,workExperience,embedding,compensation


In [40]:
vacancies = vacancies.drop(columns=["compensation.from", "compensation.to"])
vacancies.head()

,vacancy_id,company.id,compensation.currencyCode,area.id,area.regionId,employment,workSchedule,workExperience,embedding,compensation
0,v_862116,0.168631,RUR,0.357764,-0.365385,full,fullDay,noExperience,"[-2.1543527, 0.3719551, -0.5818779, 0.95323056...",-0.497627
1,v_288642,0.496334,RUR,-0.787823,-0.961538,full,fullDay,noExperience,"[-1.898529, -0.042620067, -0.34811407, 1.28335...",0.288665
2,v_1840054,0.420589,RUR,0.774704,0.500000,full,fullDay,between3And6,"[-2.688115, 0.8980422, -0.46110746, 0.91553205...",-0.999884
3,v_2346232,-0.955993,RUR,0.367460,-0.019231,full,fullDay,noExperience,"[-2.8031461, 0.6286624, -0.21013308, 1.0635489...",-0.999919
4,v_312507,0.482186,RUR,0.949808,-0.923077,full,fullDay,between1And3,"[-2.7092838, 0.6253482, -0.20391504, 1.2406433...",-0.569031


In [41]:
currencies = vacancies["compensation.currencyCode"].unique().tolist()
currencies

['RUR', None, 'USD', 'EUR', 'KZT', 'BYR', 'UZS', 'KGS', 'UAH', 'GEL', 'AZN']

In [42]:
# one-hot encoding
def ohe(sample, all_samples: list) -> np.ndarray:
    ans = np.zeros(len(all_samples), dtype=np.float32)
    ans[all_samples.index(sample)] = 1.
    return ans

ohe("RUR", currencies), ohe(None, currencies), ohe("BYR", currencies), ohe("AZN", currencies)

(array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32),
 array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32),
 array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.], dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.], dtype=float32))

In [43]:
vacancies["currency"] = vacancies["compensation.currencyCode"].progress_apply(lambda t: ohe(t, currencies))
vacancies.head()

100%|██████████| 2734129/2734129 [00:02<00:00, 1169493.89it/s]


,vacancy_id,company.id,compensation.currencyCode,area.id,area.regionId,employment,workSchedule,workExperience,embedding,compensation,currency
0,v_862116,0.168631,RUR,0.357764,-0.365385,full,fullDay,noExperience,"[-2.1543527, 0.3719551, -0.5818779, 0.95323056...",-0.497627,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,v_288642,0.496334,RUR,-0.787823,-0.961538,full,fullDay,noExperience,"[-1.898529, -0.042620067, -0.34811407, 1.28335...",0.288665,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,v_1840054,0.420589,RUR,0.774704,0.500000,full,fullDay,between3And6,"[-2.688115, 0.8980422, -0.46110746, 0.91553205...",-0.999884,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,v_2346232,-0.955993,RUR,0.367460,-0.019231,full,fullDay,noExperience,"[-2.8031461, 0.6286624, -0.21013308, 1.0635489...",-0.999919,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,v_312507,0.482186,RUR,0.949808,-0.923077,full,fullDay,between1And3,"[-2.7092838, 0.6253482, -0.20391504, 1.2406433...",-0.569031,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [44]:
employments = vacancies["employment"].unique().tolist()
employments

['full', 'part', 'project', 'probation', 'volunteer']

In [45]:
vacancies["employment"] = vacancies["employment"].progress_apply(lambda t: ohe(t, employments))
vacancies.head()

100%|██████████| 2734129/2734129 [00:02<00:00, 1156188.66it/s]


,vacancy_id,company.id,compensation.currencyCode,area.id,area.regionId,employment,workSchedule,workExperience,embedding,compensation,currency
0,v_862116,0.168631,RUR,0.357764,-0.365385,"[1.0, 0.0, 0.0, 0.0, 0.0]",fullDay,noExperience,"[-2.1543527, 0.3719551, -0.5818779, 0.95323056...",-0.497627,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,v_288642,0.496334,RUR,-0.787823,-0.961538,"[1.0, 0.0, 0.0, 0.0, 0.0]",fullDay,noExperience,"[-1.898529, -0.042620067, -0.34811407, 1.28335...",0.288665,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,v_1840054,0.420589,RUR,0.774704,0.500000,"[1.0, 0.0, 0.0, 0.0, 0.0]",fullDay,between3And6,"[-2.688115, 0.8980422, -0.46110746, 0.91553205...",-0.999884,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,v_2346232,-0.955993,RUR,0.367460,-0.019231,"[1.0, 0.0, 0.0, 0.0, 0.0]",fullDay,noExperience,"[-2.8031461, 0.6286624, -0.21013308, 1.0635489...",-0.999919,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,v_312507,0.482186,RUR,0.949808,-0.923077,"[1.0, 0.0, 0.0, 0.0, 0.0]",fullDay,between1And3,"[-2.7092838, 0.6253482, -0.20391504, 1.2406433...",-0.569031,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [46]:
schedules = vacancies["workSchedule"].unique().tolist()
schedules

['fullDay', 'flyInFlyOut', 'flexible', 'shift', 'remote']

In [47]:
vacancies["schedule"] = vacancies["workSchedule"].progress_apply(lambda t: ohe(t, schedules))
vacancies.head()

100%|██████████| 2734129/2734129 [00:02<00:00, 1334418.01it/s]


,vacancy_id,company.id,compensation.currencyCode,area.id,area.regionId,employment,workSchedule,workExperience,embedding,compensation,currency,schedule
0,v_862116,0.168631,RUR,0.357764,-0.365385,"[1.0, 0.0, 0.0, 0.0, 0.0]",fullDay,noExperience,"[-2.1543527, 0.3719551, -0.5818779, 0.95323056...",-0.497627,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0]"
1,v_288642,0.496334,RUR,-0.787823,-0.961538,"[1.0, 0.0, 0.0, 0.0, 0.0]",fullDay,noExperience,"[-1.898529, -0.042620067, -0.34811407, 1.28335...",0.288665,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0]"
2,v_1840054,0.420589,RUR,0.774704,0.500000,"[1.0, 0.0, 0.0, 0.0, 0.0]",fullDay,between3And6,"[-2.688115, 0.8980422, -0.46110746, 0.91553205...",-0.999884,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0]"
3,v_2346232,-0.955993,RUR,0.367460,-0.019231,"[1.0, 0.0, 0.0, 0.0, 0.0]",fullDay,noExperience,"[-2.8031461, 0.6286624, -0.21013308, 1.0635489...",-0.999919,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0]"
4,v_312507,0.482186,RUR,0.949808,-0.923077,"[1.0, 0.0, 0.0, 0.0, 0.0]",fullDay,between1And3,"[-2.7092838, 0.6253482, -0.20391504, 1.2406433...",-0.569031,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0]"


In [48]:
experiences = vacancies["workExperience"].unique().tolist()
experiences

['noExperience', 'between3And6', 'between1And3', 'moreThan6']

In [49]:
vacancies["experience"] = vacancies["workExperience"].progress_apply(lambda t: ohe(t, experiences))
vacancies.head()

100%|██████████| 2734129/2734129 [00:01<00:00, 1394740.82it/s]


,vacancy_id,company.id,compensation.currencyCode,area.id,area.regionId,employment,workSchedule,workExperience,embedding,compensation,currency,schedule,experience
0,v_862116,0.168631,RUR,0.357764,-0.365385,"[1.0, 0.0, 0.0, 0.0, 0.0]",fullDay,noExperience,"[-2.1543527, 0.3719551, -0.5818779, 0.95323056...",-0.497627,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0]","[1.0, 0.0, 0.0, 0.0]"
1,v_288642,0.496334,RUR,-0.787823,-0.961538,"[1.0, 0.0, 0.0, 0.0, 0.0]",fullDay,noExperience,"[-1.898529, -0.042620067, -0.34811407, 1.28335...",0.288665,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0]","[1.0, 0.0, 0.0, 0.0]"
2,v_1840054,0.420589,RUR,0.774704,0.500000,"[1.0, 0.0, 0.0, 0.0, 0.0]",fullDay,between3And6,"[-2.688115, 0.8980422, -0.46110746, 0.91553205...",-0.999884,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 1.0, 0.0, 0.0]"
3,v_2346232,-0.955993,RUR,0.367460,-0.019231,"[1.0, 0.0, 0.0, 0.0, 0.0]",fullDay,noExperience,"[-2.8031461, 0.6286624, -0.21013308, 1.0635489...",-0.999919,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0]","[1.0, 0.0, 0.0, 0.0]"
4,v_312507,0.482186,RUR,0.949808,-0.923077,"[1.0, 0.0, 0.0, 0.0, 0.0]",fullDay,between1And3,"[-2.7092838, 0.6253482, -0.20391504, 1.2406433...",-0.569031,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 1.0, 0.0]"


In [50]:
vacancies = vacancies.drop(columns=["compensation.currencyCode", "workSchedule", "workExperience"])
vacancies.head()

,vacancy_id,company.id,area.id,area.regionId,employment,embedding,compensation,currency,schedule,experience
0,v_862116,0.168631,0.357764,-0.365385,"[1.0, 0.0, 0.0, 0.0, 0.0]","[-2.1543527, 0.3719551, -0.5818779, 0.95323056...",-0.497627,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0]","[1.0, 0.0, 0.0, 0.0]"
1,v_288642,0.496334,-0.787823,-0.961538,"[1.0, 0.0, 0.0, 0.0, 0.0]","[-1.898529, -0.042620067, -0.34811407, 1.28335...",0.288665,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0]","[1.0, 0.0, 0.0, 0.0]"
2,v_1840054,0.420589,0.774704,0.500000,"[1.0, 0.0, 0.0, 0.0, 0.0]","[-2.688115, 0.8980422, -0.46110746, 0.91553205...",-0.999884,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 1.0, 0.0, 0.0]"
3,v_2346232,-0.955993,0.367460,-0.019231,"[1.0, 0.0, 0.0, 0.0, 0.0]","[-2.8031461, 0.6286624, -0.21013308, 1.0635489...",-0.999919,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0]","[1.0, 0.0, 0.0, 0.0]"
4,v_312507,0.482186,0.949808,-0.923077,"[1.0, 0.0, 0.0, 0.0, 0.0]","[-2.7092838, 0.6253482, -0.20391504, 1.2406433...",-0.569031,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 1.0, 0.0]"


In [51]:
vacancies.isna().sum()  # убедимся что не осталось nan-ов

vacancy_id       0
company.id       0
area.id          0
area.regionId    0
employment       0
embedding        0
compensation     0
currency         0
schedule         0
experience       0
dtype: int64

In [52]:
vacancies

,vacancy_id,company.id,area.id,area.regionId,employment,embedding,compensation,currency,schedule,experience
0,v_862116,0.168631,0.357764,-0.365385,"[1.0, 0.0, 0.0, 0.0, 0.0]","[-2.1543527, 0.3719551, -0.5818779, 0.95323056...",-0.497627,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0]","[1.0, 0.0, 0.0, 0.0]"
1,v_288642,0.496334,-0.787823,-0.961538,"[1.0, 0.0, 0.0, 0.0, 0.0]","[-1.898529, -0.042620067, -0.34811407, 1.28335...",0.288665,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0]","[1.0, 0.0, 0.0, 0.0]"
2,v_1840054,0.420589,0.774704,0.500000,"[1.0, 0.0, 0.0, 0.0, 0.0]","[-2.688115, 0.8980422, -0.46110746, 0.91553205...",-0.999884,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 1.0, 0.0, 0.0]"
3,v_2346232,-0.955993,0.367460,-0.019231,"[1.0, 0.0, 0.0, 0.0, 0.0]","[-2.8031461, 0.6286624, -0.21013308, 1.0635489...",-0.999919,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0]","[1.0, 0.0, 0.0, 0.0]"
4,v_312507,0.482186,0.949808,-0.923077,"[1.0, 0.0, 0.0, 0.0, 0.0]","[-2.7092838, 0.6253482, -0.20391504, 1.2406433...",-0.569031,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 1.0, 0.0]"
...,...,...,...,...,...,...,...,...,...,...
2734124,v_2484959,0.457497,0.536290,0.365385,"[1.0, 0.0, 0.0, 0.0, 0.0]","[-2.8398361, 0.82443726, -0.33765867, 0.598064...",-0.999869,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 1.0, 0.0, 0.0]","[1.0, 0.0, 0.0, 0.0]"
2734125,v_205163,0.137962,0.576216,0.326923,"[1.0, 0.0, 0.0, 0.0, 0.0]","[-2.557294, 0.5784582, -0.47409526, 0.8868624,...",-0.999788,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 1.0, 0.0]"
2734126,v_639897,0.501303,-0.499216,-0.211538,"[1.0, 0.0, 0.0, 0.0, 0.0]","[-2.929139, 0.48759758, -0.4819374, 0.99574554...",0.520782,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 1.0, 0.0]"
2734127,v_1636531,0.765753,-0.029517,0.153846,"[1.0, 0.0, 0.0, 0.0, 0.0]","[-1.7076827, 0.14056933, -0.2769349, 1.3306775...",-0.390069,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0]","[1.0, 0.0, 0.0, 0.0]"


In [53]:
# склеим все OHE-вектора

employments = np.vstack(vacancies["employment"])
currencies = np.vstack(vacancies["currency"])
schedules = np.vstack(vacancies["schedule"])
experiences = np.vstack(vacancies["experience"])

ohe_features = np.hstack([employments, currencies, schedules, experiences])
ohe_features, ohe_features.shape

(array([[1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 1., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 1., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 1., 0.]], dtype=float32),
 (2734129, 25))

In [54]:
# объединим эмбеддинги текста

embeddings = np.vstack(vacancies["embedding"])
embeddings, embeddings.shape

(array([[-2.1543527e+00,  3.7195510e-01, -5.8187789e-01, ...,
          1.0711843e-03,  9.5793498e-01, -1.8816636e+00],
        [-1.8985291e+00, -4.2620067e-02, -3.4811407e-01, ...,
         -3.9433387e-01,  9.6770871e-01, -1.7618304e+00],
        [-2.6881149e+00,  8.9804220e-01, -4.6110746e-01, ...,
          5.4468989e-02,  7.6047087e-01, -1.9440467e+00],
        ...,
        [-2.9291389e+00,  4.8759758e-01, -4.8193741e-01, ...,
         -6.6612816e-01,  5.4262793e-01, -1.5966307e+00],
        [-1.7076827e+00,  1.4056933e-01, -2.7693489e-01, ...,
         -7.3348075e-02,  7.1282929e-01, -2.0943880e+00],
        [-2.9037759e+00,  6.4310330e-01, -2.0862579e-01, ...,
         -2.6423350e-02,  5.4217106e-01, -2.0552166e+00]], dtype=float32),
 (2734129, 768))

In [55]:
company_ids = vacancies["company.id"].to_numpy(copy=True)
company_ids = np.expand_dims(company_ids, 1)
company_ids, company_ids.shape

(array([[ 0.16863085],
        [ 0.49633396],
        [ 0.42058936],
        ...,
        [ 0.50130328],
        [ 0.76575323],
        [-0.47516591]]),
 (2734129, 1))

In [56]:
area_ids = vacancies["area.id"].to_numpy(copy=True)
area_ids = np.expand_dims(area_ids, 1)
area_ids, area_ids.shape

(array([[ 0.35776415],
        [-0.78782262],
        [ 0.77470412],
        ...,
        [-0.49921574],
        [-0.02951661],
        [-0.51803793]]),
 (2734129, 1))

In [57]:
region_ids = vacancies["area.regionId"].to_numpy(copy=True)
region_ids = np.expand_dims(region_ids, 1)
region_ids, region_ids.shape

(array([[-0.36538462],
        [-0.96153846],
        [ 0.5       ],
        ...,
        [-0.21153846],
        [ 0.15384615],
        [ 0.75      ]]),
 (2734129, 1))

In [58]:
compensations = vacancies["compensation"].to_numpy(copy=True)
compensations = np.expand_dims(compensations, 1)
compensations, compensations.shape

(array([[-0.49762735],
        [ 0.28866506],
        [-0.99988405],
        ...,
        [ 0.52078189],
        [-0.39006861],
        [ 0.74151366]]),
 (2734129, 1))

In [59]:
# объединим числовые признаки в одну матрицу

numerical_features = np.hstack([company_ids, area_ids, region_ids, compensations])
numerical_features, numerical_features.shape

(array([[ 0.16863085,  0.35776415, -0.36538462, -0.49762735],
        [ 0.49633396, -0.78782262, -0.96153846,  0.28866506],
        [ 0.42058936,  0.77470412,  0.5       , -0.99988405],
        ...,
        [ 0.50130328, -0.49921574, -0.21153846,  0.52078189],
        [ 0.76575323, -0.02951661,  0.15384615, -0.39006861],
        [-0.47516591, -0.51803793,  0.75      ,  0.74151366]]),
 (2734129, 4))

In [60]:
all_features = np.hstack([numerical_features, ohe_features, embeddings])
all_features, all_features.shape

(array([[ 1.68630854e-01,  3.57764152e-01, -3.65384615e-01, ...,
          1.07118429e-03,  9.57934976e-01, -1.88166356e+00],
        [ 4.96333956e-01, -7.87822615e-01, -9.61538462e-01, ...,
         -3.94333869e-01,  9.67708707e-01, -1.76183045e+00],
        [ 4.20589364e-01,  7.74704121e-01,  5.00000000e-01, ...,
          5.44689894e-02,  7.60470867e-01, -1.94404674e+00],
        ...,
        [ 5.01303283e-01, -4.99215742e-01, -2.11538462e-01, ...,
         -6.66128159e-01,  5.42627931e-01, -1.59663069e+00],
        [ 7.65753233e-01, -2.95166120e-02,  1.53846154e-01, ...,
         -7.33480752e-02,  7.12829292e-01, -2.09438801e+00],
        [-4.75165913e-01, -5.18037930e-01,  7.50000000e-01, ...,
         -2.64233500e-02,  5.42171061e-01, -2.05521655e+00]]),
 (2734129, 797))

In [61]:
vacancies_vectors = pd.DataFrame(data={"vacancy_id": vacancies["vacancy_id"], "embedding": [e for e in all_features]})
vacancies_vectors

,vacancy_id,embedding
0,v_862116,"[0.1686308535697767, 0.3577641522886068, -0.36..."
1,v_288642,"[0.49633395599313035, -0.7878226151433053, -0...."
2,v_1840054,"[0.4205893636321263, 0.7747041209182945, 0.5, ..."
3,v_2346232,"[-0.9559931304251177, 0.36746043062883227, -0...."
4,v_312507,"[0.4821860736937589, 0.9498075003564808, -0.92..."
...,...,...
2734124,v_2484959,"[0.45749719444553993, 0.5362897476115784, 0.36..."
2734125,v_205163,"[0.13796157197098724, 0.5762155996007414, 0.32..."
2734126,v_639897,"[0.5013032831261586, -0.49921574219306997, -0...."
2734127,v_1636531,"[0.7657532331102037, -0.02951661200627409, 0.1..."


In [62]:
np.save("vacancies_features", all_features, allow_pickle=False)

In [68]:
vacancies_vectors["vacancy_id"].to_csv("vacancies.csv")

In [2]:
all_features = np.load("vacancies_features.npy")
all_features

array([[ 1.68630854e-01,  3.57764152e-01, -3.65384615e-01, ...,
         1.07118429e-03,  9.57934976e-01, -1.88166356e+00],
       [ 4.96333956e-01, -7.87822615e-01, -9.61538462e-01, ...,
        -3.94333869e-01,  9.67708707e-01, -1.76183045e+00],
       [ 4.20589364e-01,  7.74704121e-01,  5.00000000e-01, ...,
         5.44689894e-02,  7.60470867e-01, -1.94404674e+00],
       ...,
       [ 5.01303283e-01, -4.99215742e-01, -2.11538462e-01, ...,
        -6.66128159e-01,  5.42627931e-01, -1.59663069e+00],
       [ 7.65753233e-01, -2.95166120e-02,  1.53846154e-01, ...,
        -7.33480752e-02,  7.12829292e-01, -2.09438801e+00],
       [-4.75165913e-01, -5.18037930e-01,  7.50000000e-01, ...,
        -2.64233500e-02,  5.42171061e-01, -2.05521655e+00]])

In [3]:
vacancies_vectors = pd.read_csv("vacancies.csv", index_col=0)
vacancies_vectors["embedding"] = [e for e in all_features]
vacancies_vectors

,vacancy_id,embedding
0,v_862116,"[0.1686308535697767, 0.3577641522886068, -0.36..."
1,v_288642,"[0.49633395599313035, -0.7878226151433053, -0...."
2,v_1840054,"[0.4205893636321263, 0.7747041209182945, 0.5, ..."
3,v_2346232,"[-0.9559931304251177, 0.36746043062883227, -0...."
4,v_312507,"[0.4821860736937589, 0.9498075003564808, -0.92..."
...,...,...
2734124,v_2484959,"[0.45749719444553993, 0.5362897476115784, 0.36..."
2734125,v_205163,"[0.13796157197098724, 0.5762155996007414, 0.32..."
2734126,v_639897,"[0.5013032831261586, -0.49921574219306997, -0...."
2734127,v_1636531,"[0.7657532331102037, -0.02951661200627409, 0.1..."


In [5]:
dataset = datasets.Dataset.from_pandas(vacancies_vectors)
dataset

Dataset({
    features: ['vacancy_id', 'embedding', '__index_level_0__'],
    num_rows: 2734129
})

In [9]:
dataset.set_format("torch")

In [15]:
dataset.save_to_disk("/HDD/vacancies_dataset")

Saving the dataset (36/36 shards): 100%|██████████| 2734129/2734129 [00:13<00:00, 200553.13 examples/s]
